In [8]:
### ==============================================================================
# FINAL SCRIPT FOR: CNN Prediction (with Robust Edge Handling)
# ==============================================================================

# --- STEP 1: SETUP ---
!pip install -q rasterio tensorflow scikit-learn

import os
import rasterio
import numpy as np
import tensorflow as tf
from rasterio.windows import Window
import joblib

# --- STEP 2: CONFIGURATION ---
full_image_path = r"C:\Users\Danesh\Desktop\CNN_Prediction\Singrauli_Full_Image_for_Prediction-0000000000-0000009984.tif"
model_path = r"C:\Users\Danesh\Desktop\CNN_Prediction\cnn_lulc_model_final.keras"
scaler_path = r"C:\Users\Danesh\Desktop\CNN_Prediction\cnn_scaler.joblib"
output_map_path = r"C:\Users\Danesh\Desktop\CNN_Prediction\Singrauli_CNN_L.tif"


# --- STEP 3: LOAD MODELS AND METADATA ---
print("\nLoading trained model and data scaler...")
try:
    model = tf.keras.models.load_model(model_path)
    scaler = joblib.load(scaler_path)
    
    with rasterio.open(full_image_path) as src:
        out_meta = src.meta.copy()
        height = src.height
        width = src.width
    
    _, PATCH_SIZE, _, BANDS = model.input_shape
    print(f"Model loaded. Processing in {PATCH_SIZE}x{PATCH_SIZE} tiles.")
    
except Exception as e:
    print(f"An error occurred during loading: {e}")
    raise e


# --- STEP 4: PREDICT ON THE IMAGE TILE BY TILE ---
out_meta.update({"count": 1, "dtype": 'uint8', "compress": 'lzw', "nodata": 255})
print("\nStarting tile-by-tile prediction...")

try:
    with rasterio.open(output_map_path, 'w', **out_meta) as dest:
        with rasterio.open(full_image_path) as src:
            
            width_steps = range(0, src.width, PATCH_SIZE)
            height_steps = range(0, src.height, PATCH_SIZE)
            total_blocks = len(width_steps) * len(height_steps)
            current_block = 0

            for j in height_steps:
                for i in width_steps:
                    current_block += 1
                    if current_block % 1000 == 0:
                        print(f"Processing block {current_block} of {total_blocks}...")

                    # Define the window to read. It's always a full patch size.
                    read_window = Window(i, j, PATCH_SIZE, PATCH_SIZE)
                    
                    # Read the patch. `boundless=True` handles edges by filling with 0.
                    patch = src.read(window=read_window, boundless=True, fill_value=0)
                    
                    # --- Standard processing logic ---
                    bands, h_patch, w_patch = patch.shape
                    patch_flat = np.moveaxis(patch, 0, -1).reshape(-1, bands)
                    patch_flat = np.nan_to_num(patch_flat, nan=0.0)
                    patch_scaled = scaler.transform(patch_flat)
                    patch_for_pred = patch_scaled.reshape(1, h_patch, w_patch, bands)
                    pred_prob = model.predict(patch_for_pred, verbose=0)
                    pred_class = np.argmax(pred_prob, axis=-1)[0]
                    
                    # --- THIS IS THE FIX ---
                    # The `output_patch` is always the full size initially
                    output_patch = np.full((PATCH_SIZE, PATCH_SIZE), pred_class, dtype=np.uint8)
                    
                    # Calculate the actual window to write, which might be smaller at the edges.
                    write_window = Window(
                        i, j,
                        min(PATCH_SIZE, src.width - i),
                        min(PATCH_SIZE, src.height - j)
                    )
                    
                    # Trim the output patch if the write window is smaller
                    trimmed_patch = output_patch[:write_window.height, :write_window.width]
                    
                    # Write the (possibly trimmed) patch to the correct, valid window.
                    dest.write(trimmed_patch, window=write_window, indexes=1)

    print(f"\n✅ Prediction complete! Final map saved to: {output_map_path}")

except Exception as e:
    print(f"An error occurred during prediction: {e}")


Loading trained model and data scaler...
Model loaded. Processing in 10x10 tiles.

Starting tile-by-tile prediction...
Processing block 1000 of 46953...
Processing block 2000 of 46953...
Processing block 3000 of 46953...
Processing block 4000 of 46953...
Processing block 5000 of 46953...
Processing block 6000 of 46953...
Processing block 7000 of 46953...
Processing block 8000 of 46953...
Processing block 9000 of 46953...
Processing block 10000 of 46953...
Processing block 11000 of 46953...
Processing block 12000 of 46953...
Processing block 13000 of 46953...
Processing block 14000 of 46953...
Processing block 15000 of 46953...
Processing block 16000 of 46953...
Processing block 17000 of 46953...
Processing block 18000 of 46953...
Processing block 19000 of 46953...
Processing block 20000 of 46953...
Processing block 21000 of 46953...
Processing block 22000 of 46953...
Processing block 23000 of 46953...
Processing block 24000 of 46953...
Processing block 25000 of 46953...
Processing blo